# Example 4: Factories and Pipelines

This notebook demonstrates how to use factories and pipelines to create models and controllers.  Factories and pipelines provide a uniform way of working with the hyperparameter spaces.

## Set Up

As in previous examples, we will use the benchmarks package to get our system and task, and to generate the trajectory dataset.

In [1]:
import autompc as ampc
import numpy as np
from autompc.benchmarks import CartpoleSwingupBenchmark

benchmark = CartpoleSwingupBenchmark()

# Get system and task specification
system = benchmark.system
task   = benchmark.task

# Generate benchmark dataset
trajs = benchmark.gen_trajs(seed=100, n_trajs=10, traj_len=200)

Loading AutoMPC...
Cannot import SnoptWrapper
Cannot import KnitroSolver, make sure its Python interface is installed
Finished loading AutoMPC
running build_ext


## Factories

AutoMPC uses Factories to construct models, costs, and controllers from their hyperparameter configurations.

First, we instantiate the factory.  Here we use the model factory for the MLP system ID model.

In [10]:
from autompc.sysid import MLPFactory

model_factory = MLPFactory(system)

From the factory, we can then get the hyperparameter configuration space

In [12]:
model_cs = model_factory.get_configuration_space()
model_cs

Configuration space object:
  Hyperparameters:
    hidden_size_1, Type: UniformInteger, Range: [16, 256], Default: 128
    hidden_size_2, Type: UniformInteger, Range: [16, 256], Default: 128
    hidden_size_3, Type: UniformInteger, Range: [16, 256], Default: 128
    hidden_size_4, Type: UniformInteger, Range: [16, 256], Default: 128
    lr, Type: UniformFloat, Range: [1e-05, 1.0], Default: 0.001, on log-scale
    n_hidden_layers, Type: Categorical, Choices: {1, 2, 3, 4}, Default: 2
    nonlintype, Type: Categorical, Choices: {relu, tanh, sigmoid, selu}, Default: relu
  Conditions:
    hidden_size_2 | n_hidden_layers in {'2', '3', '4'}
    hidden_size_3 | n_hidden_layers in {'3', '4'}
    hidden_size_4 | n_hidden_layers in {'4'}

We can see that the MLP several hidden parameters, including categorical hyperparameters controlling the activation function and the number of hidden layers, a floating point hyperparameter controlling the learning rate, and integer hyperparameters controlling the size of each layer.

We also have conditional hyperparameter relationships.  For example, the `hidden_size_4` hyperparameter is only active when there are 4 or more hidden layers.

AutoMPC uses the [ConfigSpace](https://automl.github.io/ConfigSpace/master/) package for configurations and configuration spaces.  See the package [documentation](https://automl.github.io/ConfigSpace/master/) for more information.


From the configuration space, we can create a configuration and set it's hyperparameters as follows

In [13]:
model_cfg = model_cs.get_default_configuration()
model_cfg["n_hidden_layers"] = "2"
model_cfg["hidden_size_1"] = 32
model_cfg["hidden_size_2"] = 32

We can use the factory with a given configuration to instantiate (and train the model).

In [14]:
model = model_factory(model_cfg, trajs)

use_cuda=True
MLP Using Cuda
hidden_sizes= [32, 32]
100%|██████████| 50/50 [00:05<00:00,  8.86it/s]


This works in a similar way for costs and controllers.

In [15]:
from autompc.costs import QuadCostFactory
cost_factory = QuadCostFactory(system)
cost_cfg = cost_factory.get_configuration_space().get_default_configuration()
cost = cost_factory(cost_cfg, task, trajs)

In [16]:
from autompc.control import IterativeLQRFactory
controller_factory = IterativeLQRFactory(system)
controller_cfg = controller_factory.get_configuration_space().get_default_configuration()
controller = controller_factory(controller_cfg, task, trajs)

## Pipelines
We can use pipelines to generate the combined configuration space for a model, cost, and controller.

In [18]:
pipeline = ampc.Pipeline(system, model_factory, cost_factory, controller_factory)
pipeline.get_configuration_space()

Configuration space object:
  Hyperparameters:
    _cost:dx_F, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:dx_Q, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:omega_F, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:omega_Q, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:theta_F, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:theta_Q, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:u_R, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:x_F, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:x_Q, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _ctrlr:horizon, Type: UniformInteger, Range: [5, 25], Default: 20
    _model:hidden_size_1, Type: UniformInteger, Range: [16, 256], Default: 128
   

We can then use the pipeline to instantiate all components from the joint configuration

In [7]:
pipeline_cfg = pipeline.get_configuration_space().get_default_configuration()
controller, cost, model = pipeline(pipeline_cfg, task, trajs)

Instead of passing a factory object to the pipeline, we can instead pass an already instantiated `Model`, `Cost`, or a `Controller`.  When we do this, that component is treated as fixed and its hyperparameters are excluded from the joint configuration space.

For example, here we pass the model we trained earlier, so the joint configuration space only contains the hyperparameters for the cost and the controller

In [19]:
pipeline_fixed_model = Pipeline(system, model, cost_factory, controller_factory)
pipeline_fixed_model.get_configuration_space()

Configuration space object:
  Hyperparameters:
    _cost:dx_F, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:dx_Q, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:omega_F, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:omega_Q, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:theta_F, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:theta_Q, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:u_R, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:x_F, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:x_Q, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _ctrlr:horizon, Type: UniformInteger, Range: [5, 25], Default: 20